In [1]:
# 형태소 분석/텍스트 클리닝
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import word_tokenize
import pandas as pd 
from tqdm import tqdm
from konlpy.tag import Okt
import re

# 단어 추가를 위함
# from ckonlpy.tag import Twitter

# Soynlp 패키지 
from soynlp.noun import LRNounExtractor
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer
# import urllib.request
# from soynlp import DoublespaceLineCorpus
# from soynlp.word import WordExtractor


### 데이터 불러오기

In [113]:
df = pd.read_csv('기업데이터(30)_(전처리).csv').iloc[:, 1:]
df.columns = ["com", "date", "duty", "status", "star", "summary", "good", "bad", "expect", "doc"]
df.head()
df1 = df[["com", "date", "duty", "doc"]]

### 불용어, 숫자, 띄어쓰기, 특수문자, 의성어, 이모티콘 Cleaning
- doc -> processing

In [4]:


def clean_text(texts):
    #이모티콘 제거
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            "]+", flags=re.UNICODE)

    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')
    
    corpus = []
    for i in range(0, len(texts)):
        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(texts[i])) #remove punctuation
        review = re.sub(r'\d+','', str(texts[i]))# remove number
        review = review.lower() #lower case
        review = re.sub(r'\s+', ' ', review) #remove extra space
        # review = re.sub(r'<[^>]+>','',review) #remove Html tags
        review = re.sub(r'\s+', ' ', review) #remove spaces
        review = re.sub(r"^\s+", '', review) #remove space from start
        review = re.sub(r'\s+$', '', review) #remove space from the end
        review = re.sub(han, '', review) #remove 특수문자, 의성어
        review = re.sub(emoji_pattern, '', review) #remove 이모티콘
        corpus.append(review)
    return corpus

df1["processing"] = clean_text(df1["doc"])

C:\Users\user\AppData\Local\Temp\ipykernel_21064\3087591807.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["processing"] = clean_text(df1["doc"])


### soynlp 사용
: 기존의 형태소 분석기는 신조어나 형태소 분석기에 등록되지 않은 단어 같은 경우에는 제대로 구분하지 못하는 단점

In [12]:
pip install soynlp

     ------------------------------------- 416.8/416.8 kB 13.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### 2. 학습하기

In [15]:
doc = df['doc'][:45400]
doc.to_csv('training_file.txt', index=False, header=None, sep="\t")
#index = False : 자동으로 가장 왼쪽 컬럼에 생성된 0 부터 시작하는 인덱스 지울 때
# header = None : 헤더 이름 지울 때
# sep = "\t" : CSV 파일 기본이 comma 라서, 별도의 구분자를 두려면 변경. 예제는 탭(\t) 으로 바꿔 줌

45400

 soynlp는 학습 기반의 단어 토크나이저이므로 기존의 KoNLPy에서 제공하는 형태소 분석기들과는 달리 학습 과정을 거쳐야 합니다. 이는 전체 코퍼스로부터 응집 확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정입니다. WordExtractor.extract()를 통해서 전체 코퍼스에 대해 단어 점수표를 계산합니다.

In [54]:
trainSet = DoublespaceLineCorpus("training_file.txt")
len(trainSet)

45400

In [55]:
word_extractor = WordExtractor()
word_extractor.train(trainSet)
word_score_table = word_extractor.extract()

training was done. used memory 0.983 Gbory 0.891 Gb
all cohesion probabilities was computed. # words = 106076
all branching entropies was computed # words = 133761
all accessor variety was computed # words = 133761


### 3. SOYNLP의 응집 확률(cohesion probability)
- 응집 확률은 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도입니다. 응집 확률은 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산하여 누적곱을 한 값입니다. 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높습니다. 수식은 아래와 같습니다.

In [20]:
print(word_score_table["워라"].cohesion_forward)
print(word_score_table["워라밸"].cohesion_forward)
print(word_score_table["네카라쿠배"].cohesion_forward)

0.8509324758842444
0.6642937404743782
0.2569365176291876


### 4. SOYNLP의 브랜칭 엔트로피(branching entropy)

In [27]:
print(word_score_table["워라"].right_branching_entropy)
print(word_score_table["워라밸"].right_branching_entropy)
print(word_score_table["네카라"].right_branching_entropy)
print(word_score_table["네카라쿠배"].right_branching_entropy)
print(word_score_table["주말근무"].right_branching_entropy)
print(word_score_table["네임벨류"].right_branching_entropy)

0.700857074091836
3.247554193256744
-0.0
-0.0
2.6031790172675286
3.1256296092239726


### 5. SOYNLP의 tokenizer

In [57]:
noun_extractor = LRNounExtractor()
nouns = noun_extractor.train_extract(list(trainSet)) # list of str like


word_extractor = WordExtractor(
    min_frequency=50, # example
    min_cohesion_forward=0.05,
    min_right_branching_entropy=0.0
)

word_extractor.train(list(trainSet))
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

noun_scores = {noun:score.score for noun, score in nouns.items()}
combined_scores = {noun:score + cohesion_score.get(noun, 0)
    for noun, score in noun_scores.items()}
combined_scores.update(
    {subword:cohesion for subword, cohesion in cohesion_score.items()
    if not (subword in combined_scores)}
)

tokenizer = LTokenizer(scores=combined_scores)

[Noun Extractor] used default noun predictor; Sejong corpus predictor
[Noun Extractor] used noun_predictor_sejong
[Noun Extractor] All 2398 r features was loaded
[Noun Extractor] scanning was done (L,R) has (70019, 39778) tokens
[Noun Extractor] building L-R graph was done
[Noun Extractor] 13613 nouns are extracted
training was done. used memory 1.294 Gbory 1.135 Gb
all cohesion probabilities was computed. # words = 16330
all branching entropies was computed # words = 130545
all accessor variety was computed # words = 130545


In [106]:
train_list=list(df['doc'])
print(str(train_list[34]))
print(tokenizer.tokenize(str(train_list[34])))

좋은 인력풀에서 일해볼 수 있는 기회여서 시야를 틀수도 있지만 부바부 전체적으로 자유로운 분위기 이지만 이또한 부바부로 생긴지 오래되지 않은 부서여서 매우 자유로운편이었음 실력일는 팀원이 아닌 리더에게 잘보이는 사람에게 과업을 몰아주는 정치질도 있음. 물론 인센티브도 같이 뺏김 오래다니지 않아서 경영진까지 바라고 할게 없었다고 한다
['좋은', '인력', '풀에서', '일해볼', '수', '있는', '기회', '여서', '시야', '를', '틀수도', '있지', '만', '부바', '부', '전체적', '으로', '자유', '로운', '분위기', '이지', '만', '이또', '한', '부바', '부로', '생긴지', '오래', '되지', '않은', '부서', '여서', '매우', '자유', '로운편이었음', '실력', '일는', '팀원', '이', '아닌', '리더', '에게', '잘보', '이는', '사람', '에게', '과업', '을', '몰아주', '는', '정치', '질도', '있음.', '물론', '인센티브', '도', '같이', '뺏김', '오래', '다니지', '않아서', '경영진', '까지', '바라', '고', '할게', '없었다', '고', '한다']


In [107]:
# 빈 컬럼 만들기
df_sonlp = df1[['com', 'date', 'duty', 'doc']]
df_sonlp['soy'] = ""
# df_sonlp['soy']

# 빈 컬럼에 토큰화된 결과 추가하기
for num in range(len(df_sonlp)):
    df_sonlp['soy'][num] = tokenizer.tokenize(str(train_list[num]))
    
# 결과 출력
df_sonlp.head()


,com,date,duty,doc,soy
0,네이버,2022. 11,디자인,커리어 경력 쌓고 싶은 사람에게 추천 수평적 사무실 분위기와 복지가 다른 곳 보다는...,"[커리어, 경력, 쌓고, 싶은, 사람, 에게, 추천, 수평적, 사무실, 분위기, 와..."
1,네이버,2022. 11,전문직,자유로운 복장 분위기가 일단 편해서 좋았어요. 물론 업무는 당연히 강도가 있어야 할...,"[자유, 로운, 복장, 분위기, 가, 일단, 편해서, 좋았어요, ., 물론, 업무,..."
2,네이버,2022. 11,IT/인터넷,워라밸과 성장을 동시에 챙길 수 있는 몇 안되는 기업 앞으로도 가장 전망이 좋은 플...,"[워라밸, 과, 성장, 을, 동시, 에, 챙길, 수, 있는, 몇, 안되는, 기업, ..."
3,네이버,2022. 11,IT/인터넷,개발자가 영향력을 좀 발휘하며 일할수 있는곳. it업계 1위 개발에만 집중하며 일할...,"[개발자, 가, 영향력, 을, 좀, 발휘, 하며, 일할수, 있는곳, ., it, 업..."
4,네이버,2022. 10,IT/인터넷,아르바이트 생이었지만 꿈의 직장이란게 이런 것이구나 느낄 수 있었다 휴가 연차 등 ...,"[아르바이트, 생이었지만, 꿈의, 직장, 이란게, 이런, 것이, 구나, 느낄, 수,..."


In [231]:
# 불용어 사전
stopwords = list(pd.read_csv("stopwords_kej.csv")['words'])
stopwords.extend("이")
stopwords = set(stopwords)

# 불용어 제거 함수
def DeleteStopwords(cell):
    words = [word for word in cell if word not in stopwords]
    return words    

# 형태소 분석 적용
tokenizing_doc = []
for cell in tqdm(df_sonlp['soy']):
    tokenizing_doc.append(DeleteStopwords(cell))
df_sonlp['cleaning_soy'] = tokenizing_doc

df_sonlp.to_csv('불용어처리 결과.csv', encoding='utf-8-sig', index=False)

100%|██████████| 60573/60573 [00:00<00:00, 171296.44it/s]


In [240]:
# # ★★★★ stopwords 업데이트 꼭 하기~ ★★★★
# save_stopwords = pd.DataFrame(stopwords, columns=["words"])
# print(save_stopwords.head(2))
# save_stopwords.to_csv("stopwords_kej.csv", encoding='utf-8-sig', index=False)

  words
0    있는
1    소생
